# 일반 키즈카페 데이터 

In [ ]:
import requests
import pandas as pd
import time

# 1. 행정동 리스트 불러오기
dong_df = pd.read_excel('./서울시_법정동_행정동_매핑.xlsx')

# '행정동명' 컬럼에서 중복 없이 추출
dong_list = dong_df['행정동명'].dropna().unique().tolist()

# 2. 카카오맵 API 설정
KAKAO_API_KEY = 'e32c09e7d6b15e3397f974fd6e87238a'  # REST API KEY 입력
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

# 3. 크롤링 실행
results = []

for dong in dong_list:
    for page in range(1, 4):  
        query = f"{dong} 키즈카페"
        url = "https://dapi.kakao.com/v2/local/search/keyword.json"
        params = {"query": query, "page": page, "size": 15}
        
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"[ERROR] {dong} page {page} 요청 실패: {response.text}")
            continue

        data = response.json()
        documents = data.get('documents', [])
        if not documents:
            break 

        for doc in documents:
            results.append({
                '검색동': dong,
                '시설명': doc.get('place_name'),
                '카테고리': doc.get('category_group_name', ''),
                '도로명주소': doc.get('road_address_name', ''),
                '지번주소': doc.get('address_name', ''),
                '위도': doc.get('y'),
                '경도': doc.get('x'),
                '전화번호': doc.get('phone', ''),
                '링크': doc.get('place_url', ''),
            })

        time.sleep(0.3)  

# 4. 데이터 저장
df = pd.DataFrame(results)
df = df.drop_duplicates(subset=['시설명', '위도', '경도'])  # 중복 제거
df.to_csv('서울시_일반_키즈카페_정제최종.csv', index=False, encoding='utf-8-sig')

print("저장된 행 개수:", len(df))
